# DX 704 Week 1 Project

This week's project will build a portfolio risk and return model, and make investing recommendations for hypothetical clients.
You will collect historical data, estimate returns and risks, construct efficient frontier portfolios, and sanity check the certainty of the maximum return portfolio.

The full project description and a template notebook are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-01


Feel free to use optimization tools or libraries (such as CVXOPT or scipy.optimize) to perform any calculations required for this mini project.

### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Data

Collect historical monthly price data for the last 24 months covering 6 different stocks.
The data should cover 24 consecutive months including the last month that ended before this week's material was released on Blackboard.
To be clear, if a month ends between the Blackboard release and submitting your project, you do not need to add that month.

The six different stocks must include AAPL, SPY and TSLA.
At least one of the remaining 3 tickers must start with the same letter as your last name (e.g. professor Considine could use COIN).
This is to encourage diversity in what stocks you analyze; if you discuss this project with classmates, please make sure that you pick different tickers to differentiate your work.
Do not pick stocks with fewer than 24 consecutive months of price data.

In [1]:
# YOUR CHANGES HERE
import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf
tickers = ["AAPL", "SPY", "TSLA", "MSFT", "NVDA", "GOOGL"]
start_date = "2024-01-01"
end_date   = "2025-12-31"   # change if Blackboard release was earlier
historical_prices = None

for t in tickers:
    ticker = yf.Ticker(t)
    history = yf.Ticker(t).history(
        start=start_date,
        end=end_date,
        interval="1mo"
    )
    monthly_prices = history['Close'].resample('ME').last()

    if historical_prices is None:
        historical_prices = monthly_prices
    else:
        historical_prices = pd.concat([historical_prices, monthly_prices], axis=1)

historical_prices.columns = tickers
historical_prices.index = pd.to_datetime(historical_prices.index).strftime("%Y-%m")

historical_prices





/usr/local/python/3.12.1/lib/python3.12/site-packages/yfinance/scrapers/history.py:201: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
/usr/local/python/3.12.1/lib/python3.12/site-packages/yfinance/scrapers/history.py:201: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
/usr/local/python/3.12.1/lib/python3.12/site-packages/yfinance/scrapers/history.py:201: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
/usr/local/python/3.12.1/lib/python3.12/site-packages/yfinance/scrapers/history.py:201: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
/usr/local/python/3.12.1/lib

,AAPL,SPY,TSLA,MSFT,NVDA,GOOGL
Date,,,,,,
2024-01,182.674835,471.230652,187.289993,391.739227,61.492744,139.049393
2024-02,179.058975,495.822723,201.880005,407.563293,79.067947,137.421692
2024-03,170.092468,510.451080,175.789993,415.305878,90.305695,149.798157
2024-04,168.951767,491.391968,183.279999,384.319824,86.357948,161.559296
2024-05,190.694412,516.246338,178.080002,409.787811,109.577095,171.206406
2024-06,209.199142,532.740967,197.880005,441.994141,123.477005,180.784027
2024-07,220.581833,540.931396,232.070007,413.711243,116.969933,170.449005
2024-08,227.455139,553.570679,214.110001,412.514648,119.318932,162.340927
2024-09,231.696045,563.469910,261.630005,426.295715,121.388046,164.795197


Save the data as a TSV file named "historical_prices.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
The date should be the last trading day of the month, so it may not be the last day of the month.
For example, the last trading day of November 2024 was 2024-11-29.
The remaining columns should contain the adjusted closing prices of the corresponding stock tickers on that day.



In [2]:
# YOUR CHANGES HERE

# Index = last trading day of each month
# Columns = 6 stock tickers
from pandas.tseries.offsets import BMonthEnd
df = historical_prices.copy()
# Ensure index is datetime
df.index = pd.to_datetime(df.index)
# Convert index to LAST TRADING DAY of each month
df.index = df.index + BMonthEnd(0)
# Create required 'date' column
df = df.reset_index()
df = df.rename(columns={"index": "date"})
# Format date as YYYY-MM-DD
df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")
# Save as TSV with required headers
df.to_csv(
    "historical_prices.tsv",
    sep="\t",
    index=False
)
df

,Date,AAPL,SPY,TSLA,MSFT,NVDA,GOOGL
0,2024-01-31,182.674835,471.230652,187.289993,391.739227,61.492744,139.049393
1,2024-02-29,179.058975,495.822723,201.880005,407.563293,79.067947,137.421692
2,2024-03-29,170.092468,510.451080,175.789993,415.305878,90.305695,149.798157
3,2024-04-30,168.951767,491.391968,183.279999,384.319824,86.357948,161.559296
4,2024-05-31,190.694412,516.246338,178.080002,409.787811,109.577095,171.206406
5,2024-06-28,209.199142,532.740967,197.880005,441.994141,123.477005,180.784027
6,2024-07-31,220.581833,540.931396,232.070007,413.711243,116.969933,170.449005
7,2024-08-30,227.455139,553.570679,214.110001,412.514648,119.318932,162.340927
8,2024-09-30,231.696045,563.469910,261.630005,426.295715,121.388046,164.795197
9,2024-10-31,224.645706,560.154602,249.850006,402.568604,132.714554,170.247345


Submit "historical_prices.tsv" in Gradescope.

## Part 2: Calculate Historical Asset Returns

Calculate the historical asset returns based on the price data that you previously collected.

In [3]:
# YOUR CHANGES HERE
historical_returns = historical_prices.pct_change().dropna()
estimated_returns = historical_returns.mean().rename("return")

Save the data as a TSV file named "historical_returns.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
Each row should have the date at the end of the month and the corresponding *relative* price changes.
For example, if the previous price was \$100 and the new price is \$110, the return value should be 0.10.
There should only be 23 rows of data in this file, since they are computed as the differences of 24 prices.

In [4]:
# YOUR CHANGES HERE
# Calculate monthly relative returns
historical_returns = historical_prices.pct_change().dropna()
# Ensure index is datetime (last trading day of month)
historical_returns.index = pd.to_datetime(historical_returns.index)
# Reset index to create 'date' column
historical_returns = historical_returns.reset_index()
historical_returns = historical_returns.rename(columns={"index": "Date"})
# Format date as YYYY-MM-DD
historical_returns["Date"] = historical_returns["Date"].dt.strftime("%Y-%m-%d")
# Save as TSV
historical_returns.to_csv(
    "historical_returns.tsv",
    sep="\t",
    index=False
)
historical_returns.head()

,Date,AAPL,SPY,TSLA,MSFT,NVDA,GOOGL
0,2024-02-01,-0.019794,0.052187,0.077901,0.040394,0.285809,-0.011706
1,2024-03-01,-0.050076,0.029503,-0.129235,0.018997,0.142128,0.090062
2,2024-04-01,-0.006706,-0.037338,0.042608,-0.074610,-0.043715,0.078513
3,2024-05-01,0.128691,0.050580,-0.028372,0.066268,0.268871,0.059713
4,2024-06-01,0.097039,0.031951,0.111186,0.078593,0.126851,0.055942


Submit "historical_returns.tsv" in Gradescope.

## Part 3: Estimate Returns

Estimate the expected returns for each asset using the previously calculated return data.
Just compute the average (mean) return for each asset over your data set; do not use other estimators that have been mentioned.
This will serve as your estimate of expected return for each asset.

In [5]:
expected_returns = estimated_returns.sort_values()

Save the estimated returns in a TSV file named "estimated_returns.tsv" and include a header row with the column names "asset" and "estimated_return".

In [6]:
# YOUR CHANGES HERE

estimated_returns_df = expected_returns.reset_index()
estimated_returns_df.columns = ["asset", "estimated_return"]

# Save as TSV
estimated_returns_df.to_csv(
    "estimated_returns.tsv",
    sep="\t",
    index=False
)

estimated_returns_df

,asset,estimated_return
0,MSFT,0.010854
1,SPY,0.016532
2,AAPL,0.019061
3,GOOGL,0.038990
4,TSLA,0.049640
5,NVDA,0.055531


Submit "estimated_returns.tsv" in Gradescope.

## Part 4: Estimate Risk

Estimate the covariance matrix for the asset returns to understand how the assets move together.

In [7]:
# YOUR CHANGES HERE
covariance_matrix = historical_returns.cov()
covariance_matrix

...

ValueError: could not convert string to float: '2024-02-01'

Save the estimated covariances to a TSV file named "estimated_covariance.tsv".
The header row should have a blank column name followed by the names of the assets.
Each data row should start with the name of an asset for that row, and be followed by the individual covariances corresponding to that row and column's assets.
(This is the format of pandas's `to_csv` method with `sep="\t"` when used on a covariance matrix as computed in the examples.)

In [ ]:
# YOUR CHANGES HERE

covariance_matrix.to_csv(
    "estimated_covariance.tsv",
    sep="\t"
)

Submit "estimated_covariance.tsv" in Gradescope.

## Part 5: Construct the Maximum Return Portfolio

Compute the maximum return portfolio based on your previously estimated risks and returns.

In [ ]:
# YOUR CHANGES HERE

mu = expected_returns.values              # (6,)
assets = expected_returns.index.tolist()  # asset names in same order
n = len(mu)
w = cp.Variable(n)

objective = cp.Maximize(mu @ w)
constraints = [
    cp.sum(w) == 1,
    w >= 0
]

problem = cp.Problem(objective, constraints)
problem.solve()

max_return_weights = pd.Series(w.value, index=assets, name="weight")
max_return_weights
Sigma = covariance_matrix.values
portfolio_expected_return = float(mu @ max_return_weights.values)
portfolio_variance = float(max_return_weights.values.T @ Sigma @ max_return_weights.values)
portfolio_volatility = float(np.sqrt(portfolio_variance))
portfolio_expected_return, portfolio_volatility
max_return_weights.sort_values(ascending=False)



MSFT     1.369651e-09
SPY      1.601697e-09
AAPL     1.729488e-09
GOOGL    3.375317e-09
TSLA     8.510915e-10
NVDA     1.000000e+00
Name: weight, dtype: float64

Save the maximum return portfolio in a TSV file named "maximum_return.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [ ]:
# YOUR CHANGES HERE

maximum_return_df = max_return_weights.reset_index()
maximum_return_df.columns = ["asset", "allocation"]

# (Optional but recommended) enforce numerical cleanliness
maximum_return_df["allocation"] = maximum_return_df["allocation"].round(6)

# Sanity check: allocations sum to 1
print("Allocation sum:", maximum_return_df["allocation"].sum())

# Save as TSV
maximum_return_df.to_csv(
    "maximum_return.tsv",
    sep="\t",
    index=False
)

maximum_return_df

Allocation sum: 1.0


,asset,allocation
0,MSFT,0.0
1,SPY,0.0
2,AAPL,0.0
3,GOOGL,0.0
4,TSLA,0.0
5,NVDA,1.0


Submit "maximum_return.tsv" in Gradescope.

## Part 6: Construct the Minimum Risk Portfolio

Compute the minimum risk portfolio based on your previously estimated risks.

In [ ]:
# YOUR CHANGES HERE

...

Ellipsis

Save the minimum risk portfolio in a TSV file named "minimum_risk.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [ ]:
# YOUR CHANGES HERE

...

Ellipsis

Submit "minimum_risk.tsv" in Gradescope.

## Part 7: Build Efficient Frontier Portfolios

Compute 101 portfolios along the mean-variance efficient frontier with evenly spaced estimated returns.
The first portfolio should be the minimum risk portfolio from part 4, and the last portfolio should be the maximum return portfolio from part 3.
The estimated return of each portfolio should be higher than the previous by one percent of the difference between the first and last portfolios.
That is, the estimated return of the portfolios should be similar to `np.linspace(min_risk_return, max_return, 101)`.


In [ ]:
# YOUR CHANGES HERE

...

Ellipsis

Save the portfolios in a TSV file named "efficient_frontier.tsv".
The header row should have columns "index", "return", "risk", and all the asset tickers.
Each data row should have the portfolio index (0-100), the estimated return of the portfolio, the estimated standard deviation (not variance) of the portfolio, and all the asset allocations (which should sum to one).

In [ ]:
# YOUR CHANGES HERE

...

Ellipsis

Submit "efficient_frontier.tsv" in Gradescope.

## Part 8: Check Maximum Return Portfolio Stability

Check the stability of the maximum return portfolio by resampling the estimated risk/return model.

Repeat 1000 times -
1. Use `np.random.multivariate_normal` to generate 23 return samples using your previously estimated risks and returns.
2. Estimate the return of each asset using that resampled return history.
3. Check which asset had the highest return in those resampled estimates.

This procedure is a reduced and simplified version of the Michaud resampled efficient frontier procedure that takes uncertainty in the risk model into account.

In [ ]:
# YOUR CHANGES HERE

...

Ellipsis

Save a file "max_return_probabilities.tsv" with the distribution of highest return assets.
The header row should have columns "asset" and "probability".
There should be a data row for each asset and its sample probability of having the highest return based on those 1000 resampled estimates.


In [ ]:
# YOUR CHANGES HERE

...

Ellipsis

Submit "max_return_probabilities.tsv" in Gradescope.

## Part 9: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgments are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 10: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.